## Gun and Knife Detection

## Initialization

The following cells will adjust the Makefile to enable openCV and GPU for darknet and then build darknet.

In [1]:
# To clone darknet repo
!git clone https://github.com/AlexeyAB/darknet
# We change the makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
# To verify CUDA
!/usr/local/cuda/bin/nvcc --version
# To Print GPU info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Cloning into 'darknet'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 14333 (delta 0), reused 0 (delta 0), pack-reused 14330
Receiving objects: 100% (14333/14333), 12.88 MiB | 2.96 MiB/s, done.
Resolving deltas: 100% (9777/9777), done.


In [5]:
# make darknet (This builds darknet so that we can then use the darknet executable file to run or train object detectors)
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cuda/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
/bin/sh: pkg-config: command not found
In file included from ./src/image.h:3,
                 from ./src/image_opencv.h:4,
                 from ./src/image_opencv.cpp:1:
include/darknet.h:41:10: fatal error: cuda_runtime.h: No such file or directory
   41 | #include <cuda_runtime.h>
      |          ^~~~~~~~~~~~~~~~
compilation terminated.
make: *** [obj/image_opencv.o] Error 1


These three functions are helper functions that allow us to show the image in our Colab Notebook after running our detections, as well as upload and download images to and from our Cloud VM.

In [2]:
# Define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
# Mounting Google Drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# This creates a symbolic link so that the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [ ]:
# cd back into the darknet folder to run detections
%cd darknet

In [ ]:
# Copy over both datasets into the root directory of the Colab VM
!cp /mydrive/yolov4/obj.zip ../
!cp /mydrive/yolov4/test.zip ../

In [ ]:
# Unzip the datasets and their contents so that they are now in /darknet/data/ folder
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/

# Configuring Files for Training
This step involves properly configuring our custom .cfg, obj.data, obj.names, train.txt and test.txt files.

## i) Cfg File

In [ ]:
# Upload the custom .cfg back to cloud VM from Google Drive
!cp /mydrive/yolov4/yolov4-obj.cfg ./cfg

## ii) obj.names and obj.data 

In [1]:
# Upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /mydrive/yolov4/obj.names ./data
!cp /mydrive/yolov4/obj.data  ./data

'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.


## iii) Generating train.txt and test.txt
train.txt and test.txt files hold the relative paths to all our training images and valdidation images.

In [ ]:
# Upload the generate_train.py and generate_test.py script to cloud VM from Google Drive
!cp /mydrive/yolov4/generate_train.py ./
!cp /mydrive/yolov4/generate_test.py ./

In [ ]:
!python generate_train.py
!python generate_test.py

In [ ]:
# verify that the newly generated train.txt and test.txt can be seen in our darknet/data folder
!ls data/

This step downloads the weights for the convolutional layers of the YOLOv4 network

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# Training
We will use this command to train the network on our dataset
```
!./darknet detector train <path to obj.data> <path to custom config> yolov4.conv.137 -dont_show -map
```

In [ ]:
!pip install pyngrok
from pyngrok import ngrok # Open a HTTP tunnel on port 8090
public_url = ngrok.connect(port = '8090')

public_url

In [ ]:
# Training our custom detector
# %%capture
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map -mjpeg_port 8090

In [2]:
#imShow('chart.png')

If for some reason we get an error or our Colab goes idle during training, we have not lost your partially trained model and weights. Every 100 iterations a weights file called **yolov4-obj_last.weights** is saved to **mydrive/yolov4/backup/** folder

We can kick off training from our last saved weights file so that we don't have to restart!

In [ ]:
# Kick off training from where it last saved
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights -dont_show -mjpeg_port 8090 -map

# Checking the Mean Average Precision (mAP) of our Model
We can run the following command on any of the saved weights from the training to see the mAP value for that specific weight's file.

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights
#this is for best weight file

## Running our Custom Object Detector

In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=32/subdivisions=1/' yolov4-obj.cfg #subdivision=32 currently
%cd ..

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/1.jpg -dont_show -thresh 0.3
imShow('predictions.jpg')

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/5.jpg -dont_show
imShow('predictions.jpg')

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/6.jpg -dont_show
imShow('predictions.jpg')

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/7.jpg -dont_show
imShow('predictions.jpg')

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/8.jpg -dont_show
imShow('predictions.jpg')

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/9.jpg -dont_show
imShow('predictions.jpg')

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/10.jpg -dont_show
imShow('predictions.jpg')

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/11.jpg -dont_show
imShow('predictions.jpg')

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/vid1.mp4 -out_filename result1.mp4 -ext_output -dont_show

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/vid2.mp4 -out_filename result2.mp4 -ext_output -dont_show

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/vid3.mp4 -out_filename result3.mp4 -ext_output -dont_show

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/vid4.mp4 -out_filename result4.mp4 -ext_output -dont_show

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/vid5.mp4 -out_filename result5.mp4 -ext_output -dont_show

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/vid6.mp4 -out_filename result6.mp4 -ext_output -dont_show

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/vid7.mp4 -out_filename result7.mp4 -ext_output -dont_show

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/vid8.mp4 -out_filename result8.mp4 -ext_output -dont_show

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/vid9.mp4 -out_filename result9.mp4 -ext_output -dont_show

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/vid8.mp4 -out_filename result10.mp4 -ext_output -dont_show -thresh 0.4
#best with threshold

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/vid9.mp4 -out_filename result11.mp4 -ext_output -dont_show -thresh 0.4
#best with threshold

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights /mydrive/images/vid2.mp4 -out_filename result12.mp4 -ext_output -dont_show
#last

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights /mydrive/images/vid8.mp4 -out_filename result13.mp4 -ext_output -dont_show
#last

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights /mydrive/images/vid9.mp4 -out_filename result14.mp4 -ext_output -dont_show
#last

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights /mydrive/images/vid8.mp4 -out_filename result15.mp4 -ext_output -dont_show -thresh 0.4
#last with threshold

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights /mydrive/images/vid9.mp4 -out_filename result16.mp4 -ext_output -dont_show -thresh 0.4
#last with threshold

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/vid8.mp4 -out_filename result17.mp4 -ext_output -dont_show -thresh 0.3
#best with threshold 0.3

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/vid9.mp4 -out_filename result18.mp4 -ext_output -dont_show -thresh 0.3
#best with threshold 0.3

In [ ]:
download('result1.mp4')

In [ ]:
download('result2.mp4')

In [ ]:
download('result3.mp4')

In [ ]:
download('result4.mp4')

In [ ]:
download('result5.mp4')

In [ ]:
download('result6.mp4')

In [ ]:
download('result7.mp4')

In [ ]:
download('result8.mp4')

In [ ]:
download('result9.mp4')

In [ ]:
download('result10.mp4')

In [ ]:
download('result11.mp4')

In [ ]:
download('result12.mp4')

In [ ]:
download('result13.mp4')

In [ ]:
download('result14.mp4')

In [ ]:
download('result15.mp4')

In [ ]:
download('result16.mp4')

In [ ]:
download('result17.mp4')

In [ ]:
download('result18.mp4')

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/newvid2.mp4 -out_filename result19.mp4 -ext_output -dont_show -thresh 0.3
#best with threshold 0.3

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/newvid3.mp4 -out_filename result20.mp4 -ext_output -dont_show -thresh 0.3
#best with threshold 0.3

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/newvid4.mp4 -out_filename result21.mp4 -ext_output -dont_show -thresh 0.3
#best with threshold 0.3

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_best.weights /mydrive/images/vid2.mp4 -out_filename result22.mp4 -ext_output -dont_show -thresh 0.3
#best with threshold 0.3

In [ ]:
download('result19.mp4')

In [ ]:
download('result20.mp4')

In [ ]:
download('result21.mp4')

In [ ]:
download('result22.mp4')

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights -c 0